In [1]:
import os
import cv2
import random

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras.applications as app
import keras.utils as utils

from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as KB
from keras.models import Model
from keras.layers import Dense, MaxPooling2D, GlobalAveragePooling2D

In [2]:
seed_num = 42
np.random.seed(seed_num)
random.seed(seed_num)
tf.random.set_seed(seed_num)

In [3]:
img_size = 224

In [4]:
def make_data(patha = None, pathb = None, size = (224,224)):
    data = []
    y = []
    j = 0
    
    if len(os.listdir(pathb)) != 0:
        for i in os.listdir(pathb):
            if i[-3:] == 'jpg':
                data_dogs = cv2.imread(pathb + '/'+str(i))
                res = cv2.resize(data_dogs, size) 
                y.append(0)
                data.append(res)
    
    if len(os.listdir(patha)) != 0:
        for i in os.listdir(patha):
            if i[-3:] == 'jpg':
                data_cats = cv2.imread(patha + '/'+str(i))
                res = cv2.resize(data_cats, size) 
                y.append(1)
                data.append(res)
    
    y = np.array(y)
    data = np.array(data)
    
    return data, y

In [5]:
X_train,y_train = make_data(pathb='training_set/dogs', patha='training_set/cats')

In [6]:
X_train.shape, y_train.shape

((8005, 224, 224, 3), (8005,))

In [7]:
y_category_train = utils.to_categorical(y_train)

In [8]:
y_category_train.shape

(8005, 2)

In [9]:
train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

train_datagen.fit(X_train)

In [10]:
def clear():
    KB.clear_session()

clear()

In [11]:
base_model = app.ResNet50(include_top = False, input_shape = (img_size, img_size, 3),weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

In [12]:
x = base_model.output
pool = GlobalAveragePooling2D()(x)

In [13]:
output = Dense(2,activation = 'softmax')(pool)

In [14]:
model = Model(inputs = base_model.input, outputs = output)

In [15]:
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [16]:
y_category_train.shape, X_train.shape

((8005, 2), (8005, 224, 224, 3))

In [17]:
model

In [18]:
model.fit(X_train, y_category_train, epochs = 5)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
251/251 [==============================] - 14s 55ms/step - loss: 0.0881 - accuracy: 0.9669
Epoch 2/5
251/251 [==============================] - ETA: 0s - loss: 0.0413 - accuracy: 0.98 - 14s 54ms/step - loss: 0.0412 - accuracy: 0.9844
Epoch 3/5
251/251 [==============================] - 14s 54ms/step - loss: 0.0307 - accuracy: 0.9879
Epoch 4/5
251/251 [==============

In [19]:
X_test,y_test = make_data(pathb = 'test_set/dogs', patha='test_set/cats')

In [20]:
y_category_test = utils.to_categorical(y_test)

In [21]:
model.evaluate(X_test, y_category_test)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
64/64 [==============================] - 4s 56ms/step - loss: 0.0389 - accuracy: 0.9876


[0.0389191173017025, 0.9876421093940735]

In [22]:
model.save('cat-dog-ResNet50.h5')